In [0]:
# https://keras.io/
!pip install -q keras
import numpy as np

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 64
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Convert the 32x32x3 images to 24x24x3 using numpy resize function.

In [55]:
x_train_24 = []
x_test_24 = []

for img in x_train:
  x_train_24.append(np.resize(img,(24,24,3)))
  
for img in x_test:
  x_test_24.append(np.resize(img,(24,24,3)))

x_train_24 = np.array(x_train_24)
x_test_24 = np.array(x_test_24)
print(x_train_24[0].shape)
resizing the images to 24X24

24 24 3


In [0]:
def add_denseblock(input, num_filter = 64, dropout_rate = 0.35):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.35):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

## Network for 24x24 is slightly different than network for 32x32. 
## Creating the network for 24x24

In [0]:
num_filter = 64
dropout_rate = 0.3
l = 12
input = Input(shape=(24, 24, 3))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [71]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv2d_209 (Conv2D)             (None, 24, 24, 64)   1728        input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_209 (BatchN (None, 24, 24, 64)   256         conv2d_209[0][0]                 
__________________________________________________________________________________________________
activation_209 (Activation)     (None, 24, 24, 64)   0           batch_normalization_209[0][0]    
__________________________________________________________________________________________________
conv2d_210

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

## Train the model for 24x24x3 images for 15 epochs

In [74]:
#trying to fit a model for small image size
model.fit(x_train_24, y_train,
                    batch_size=64,
                    epochs=15,
                    verbose=1,
                    validation_data=(x_test_24, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
43776/50000 [=========================>....] - ETA: 45s - loss: 1.5142 - acc: 0.441250000/50000 [==============================] - 387s 8ms/step - loss: 1.4776 - acc: 0.4558 - val_loss: 1.5257 - val_acc: 0.4740
Epoch 2/10
26240/50000 [==============>...............] - ETA: 2:37 - loss: 1.1053 - acc: 0.605450000/50000 [==============================] - 355s 7ms/step - loss: 1.0588 - acc: 0.6207 - val_loss: 3.1899 - val_acc: 0.3719
Epoch 3/10
19584/50000 [==========>...................] - ETA: 3:21 - loss: 0.9253 - acc: 0.672350000/50000 [==============================] - 355s 7ms/step - loss: 0.8917 - acc: 0.6843 - val_loss: 1.2361 - val_acc: 0.6404
Epoch 4/10
17024/50000 [=========>....................] - ETA: 3:37 - loss: 0.8007 - acc: 0.717650000/50000 [==============================] - 355s 7ms/step - loss: 0.7801 - acc: 0.7256 - val_loss: 1.6979 - val_acc: 0.5537
Epoch 5/10
16064/50000 [========>.....................] - E

In [17]:
# Test the model
score = model.evaluate(x_test_24, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 30s 3ms/step
Test loss: 0.7400615870952606
Test accuracy: 0.7899


In Order to run the model on a different resolution image, the number of parameters or the weights on the last layer of model differs. So, we pop the last layers in order to give space for a higher resolution image. Therefore, this makes the architecture slightly different from the architecture for 24x24.

In [18]:
model.layers.pop()
model.layers.pop()

After popping out we save the model weights. Here, the weights for last layers are also popped out. Therefore, we only store weights except the popped out layers

In [19]:
#save the weights from the model above
model.save_weights("weights_after_pop.hdf5")
print("Saved model to disk")

Saved model to disk


## Here we construct the network by adding a simple batch normalization layer with relu and average pooling.

In [0]:
num_filter = 64
dropout_rate = 0.2
l = 15
input = Input(shape=(32, 32, 3,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
BatchNorm = BatchNormalization()(Last_Block)
relu = Activation('relu')(BatchNorm)
AP = AveragePooling2D()(relu)
output = Dense(num_classes, activation='softmax')(Flatten()(AP))

In [25]:
model=Model(inputs=[model.input], outputs=[output])
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
             metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 64)   256         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 64)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

Load the previous saved weights

In [0]:
model.load_weights('weights_after_pop.hdf5')

In [0]:
#creating checkpoints to save the weights after every 10 epochs
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Train for 5 epochs

In [27]:
#Load the weights from above and then run the model on train data of actual size, for remaining nmber of epochs
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=20,
                    verbose=1,
                    initial_epoch=15,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 16/20
21696/50000 [============>.................] - ETA: 6:03 - loss: 0.5710 - acc: 0.8073

50000/50000 [==============================] - 670s 13ms/step - loss: 0.5313 - acc: 0.8189 - val_loss: 0.6138 - val_acc: 0.8212

Epoch 00016: val_acc improved from -inf to 0.82120, saving model to weights-improvement-16-0.82.hdf5
Epoch 17/20
 2368/50000 [>.............................] - ETA: 9:46 - loss: 0.4512 - acc: 0.8471

49984/50000 [============================>.] - ETA: 0s - loss: 0.4475 - acc: 0.8463

50000/50000 [==============================] - 657s 13ms/step - loss: 0.4477 - acc: 0.8462 - val_loss: 0.9885 - val_acc: 0.7457

Epoch 00017: val_acc did not improve from 0.82120
Epoch 18/20
 8832/50000 [====>.........................] - ETA: 8:26 - loss: 0.4356 - acc: 0.8511

49984/50000 [============================>.] - ETA: 0s - loss: 0.4151 - acc: 0.8565

50000/50000 [==============================] - 657s 13ms/step - loss: 0.4150 - acc: 0.8565 - val_loss: 0.6862 - val_acc: 0.8185

Epoch 00018: val_acc did not improve from 0.82120
Epoch 19/20
 8832/50000 [====>.........................] - ETA: 8:26 - loss: 0.3855 - acc: 0.8665

49984/50000 [============================>.] - ETA: 0s - loss: 0.3905 - acc: 0.8655

50000/50000 [==============================] - 656s 13ms/step - loss: 0.3906 - acc: 0.8655 - val_loss: 2.2204 - val_acc: 0.6107

Epoch 00019: val_acc did not improve from 0.82120
Epoch 20/20
 8832/50000 [====>.........................] - ETA: 8:26 - loss: 0.3661 - acc: 0.8698

49984/50000 [============================>.] - ETA: 0s - loss: 0.3627 - acc: 0.8750

50000/50000 [==============================] - 656s 13ms/step - loss: 0.3629 - acc: 0.8750 - val_loss: 0.7225 - val_acc: 0.8035

Epoch 00020: val_acc did not improve from 0.82120


Train for 10 epochs from 20 to 30

In [30]:
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=30,
                    verbose=1,
                    initial_epoch=20,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 21/30
43456/50000 [=========================>....] - ETA: 1:05 - loss: 0.2597 - acc: 0.9100

50000/50000 [==============================] - 539s 11ms/step - loss: 0.2625 - acc: 0.9097 - val_loss: 0.6962 - val_acc: 0.8111

Epoch 00021: val_acc did not improve from 0.82120
Epoch 22/30
21056/50000 [===========>..................] - ETA: 4:47 - loss: 0.2495 - acc: 0.9126

50000/50000 [==============================] - 536s 11ms/step - loss: 0.2586 - acc: 0.9092 - val_loss: 0.5792 - val_acc: 0.8449

Epoch 00022: val_acc improved from 0.82120 to 0.84490, saving model to weights-improvement-22-0.84.hdf5
Epoch 23/30
11776/50000 [======>.......................] - ETA: 6:20 - loss: 0.2357 - acc: 0.9170

50000/50000 [==============================] - 535s 11ms/step - loss: 0.2507 - acc: 0.9121 - val_loss: 0.5047 - val_acc: 0.8641

Epoch 00023: val_acc improved from 0.84490 to 0.86410, saving model to weights-improvement-23-0.86.hdf5
Epoch 24/30
 9408/50000 [====>.........................] - ETA: 6:41 - loss: 0.2383 - acc: 0.9183

50000/50000 [==============================] - 535s 11ms/step - loss: 0.2441 - acc: 0.9151 - val_loss: 0.7602 - val_acc: 0.8236

Epoch 00024: val_acc did not improve from 0.86410
Epoch 25/30
10560/50000 [=====>........................] - ETA: 6:32 - loss: 0.2330 - acc: 0.9190

...

50000/50000 [==============================] - 534s 11ms/step - loss: 0.2200 - acc: 0.9230 - val_loss: 0.4576 - val_acc: 0.8731

Epoch 00026: val_acc improved from 0.86410 to 0.87310, saving model to weights-improvement-26-0.87.hdf5
Epoch 27/30
 9216/50000 [====>.........................] - ETA: 6:45 - loss: 0.1947 - acc: 0.9349

50000/50000 [==============================] - 534s 11ms/step - loss: 0.2149 - acc: 0.9251 - val_loss: 0.8092 - val_acc: 0.8145

Epoch 00027: val_acc did not improve from 0.87310
Epoch 28/30
10560/50000 [=====>........................] - ETA: 6:31 - loss: 0.1945 - acc: 0.9281

50000/50000 [==============================] - 534s 11ms/step - loss: 0.2065 - acc: 0.9259 - val_loss: 0.5716 - val_acc: 0.8604

Epoch 00028: val_acc did not improve from 0.87310
Epoch 29/30
10944/50000 [=====>........................] - ETA: 6:27 - loss: 0.1927 - acc: 0.9331

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1970 - acc: 0.9301 - val_loss: 0.6385 - val_acc: 0.8426

Epoch 00029: val_acc did not improve from 0.87310
Epoch 30/30
11072/50000 [=====>........................] - ETA: 6:26 - loss: 0.1750 - acc: 0.9395

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1876 - acc: 0.9331 - val_loss: 0.5448 - val_acc: 0.8675

Epoch 00030: val_acc did not improve from 0.87310


Train for 10 epochs from 30 to 40

In [33]:
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=40,
                    verbose=1,
                    initial_epoch=30,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 31/40
43456/50000 [=========================>....] - ETA: 1:04 - loss: 0.1856 - acc: 0.9355

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1845 - acc: 0.9355 - val_loss: 0.6562 - val_acc: 0.8438

Epoch 00031: val_acc did not improve from 0.87310
Epoch 32/40
21056/50000 [===========>..................] - ETA: 4:47 - loss: 0.1621 - acc: 0.9424

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1761 - acc: 0.9377 - val_loss: 0.6367 - val_acc: 0.8533

Epoch 00032: val_acc did not improve from 0.87310
Epoch 33/40
14208/50000 [=======>......................] - ETA: 5:55 - loss: 0.1629 - acc: 0.9411

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1679 - acc: 0.9398 - val_loss: 0.5162 - val_acc: 0.8719

Epoch 00033: val_acc did not improve from 0.87310
Epoch 34/40
12096/50000 [======>.......................] - ETA: 6:16 - loss: 0.1593 - acc: 0.9431

50000/50000 [==============================] - 533s 11ms/step - loss: 0.1646 - acc: 0.9412 - val_loss: 0.6398 - val_acc: 0.8616

Epoch 00034: val_acc did not improve from 0.87310
Epoch 35/40
11392/50000 [=====>........................] - ETA: 6:22 - loss: 0.1463 - acc: 0.9498

...

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1542 - acc: 0.9449 - val_loss: 0.5475 - val_acc: 0.8734

Epoch 00036: val_acc improved from 0.87310 to 0.87340, saving model to weights-improvement-36-0.87.hdf5
Epoch 37/40
 9280/50000 [====>.........................] - ETA: 6:44 - loss: 0.1380 - acc: 0.9509

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1469 - acc: 0.9479 - val_loss: 0.7561 - val_acc: 0.8396

Epoch 00037: val_acc did not improve from 0.87340
Epoch 38/40
10560/50000 [=====>........................] - ETA: 6:31 - loss: 0.1448 - acc: 0.9473

50000/50000 [==============================] - 534s 11ms/step - loss: 0.1441 - acc: 0.9485 - val_loss: 0.5626 - val_acc: 0.8719

Epoch 00038: val_acc did not improve from 0.87340
Epoch 39/40
10944/50000 [=====>........................] - ETA: 6:28 - loss: 0.1290 - acc: 0.9513

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1379 - acc: 0.9500 - val_loss: 0.5586 - val_acc: 0.8719

Epoch 00039: val_acc did not improve from 0.87340
Epoch 40/40
11072/50000 [=====>........................] - ETA: 6:27 - loss: 0.1278 - acc: 0.9548

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1331 - acc: 0.9521 - val_loss: 0.6553 - val_acc: 0.8647

Epoch 00040: val_acc did not improve from 0.87340


Train for 10 epochs from 40 to 50

In [34]:
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    initial_epoch=40,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 41/50
43456/50000 [=========================>....] - ETA: 1:05 - loss: 0.1265 - acc: 0.9539

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1288 - acc: 0.9530 - val_loss: 0.7632 - val_acc: 0.8383

Epoch 00041: val_acc did not improve from 0.87340
Epoch 42/50
21056/50000 [===========>..................] - ETA: 4:47 - loss: 0.1189 - acc: 0.9572

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1265 - acc: 0.9551 - val_loss: 0.4971 - val_acc: 0.8860

Epoch 00042: val_acc improved from 0.87340 to 0.88600, saving model to weights-improvement-42-0.89.hdf5
Epoch 43/50
11776/50000 [======>.......................] - ETA: 6:20 - loss: 0.1188 - acc: 0.9593

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1222 - acc: 0.9562 - val_loss: 0.5528 - val_acc: 0.8729

Epoch 00043: val_acc did not improve from 0.88600
Epoch 44/50
11328/50000 [=====>........................] - ETA: 6:25 - loss: 0.1143 - acc: 0.9577

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1180 - acc: 0.9579 - val_loss: 0.6221 - val_acc: 0.8741

Epoch 00044: val_acc did not improve from 0.88600
Epoch 45/50
11200/50000 [=====>........................] - ETA: 6:25 - loss: 0.1136 - acc: 0.9596

...

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1169 - acc: 0.9589 - val_loss: 0.5322 - val_acc: 0.8803

Epoch 00046: val_acc did not improve from 0.89110
Epoch 47/50
10560/50000 [=====>........................] - ETA: 6:32 - loss: 0.1051 - acc: 0.9607

50000/50000 [==============================] - 536s 11ms/step - loss: 0.1099 - acc: 0.9605 - val_loss: 0.6722 - val_acc: 0.8609

Epoch 00047: val_acc did not improve from 0.89110
Epoch 48/50
10944/50000 [=====>........................] - ETA: 6:28 - loss: 0.1000 - acc: 0.9645

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1071 - acc: 0.9608 - val_loss: 0.5482 - val_acc: 0.8882

Epoch 00048: val_acc did not improve from 0.89110
Epoch 49/50
11072/50000 [=====>........................] - ETA: 6:27 - loss: 0.0959 - acc: 0.9648

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1052 - acc: 0.9622 - val_loss: 0.5199 - val_acc: 0.8801

Epoch 00049: val_acc did not improve from 0.89110
Epoch 50/50
11136/50000 [=====>........................] - ETA: 6:26 - loss: 0.1007 - acc: 0.9643

50000/50000 [==============================] - 535s 11ms/step - loss: 0.1031 - acc: 0.9629 - val_loss: 0.5009 - val_acc: 0.8924

Epoch 00050: val_acc improved from 0.89110 to 0.89240, saving model to weights-improvement-50-0.89.hdf5


In [35]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 43s 4ms/step
Test loss: 0.5009491809621454
Test accuracy: 0.8924


In [0]:
from google.colab import files

files.download('weights-improvement-50-0.89.hdf5')